# Standard Imports

In [0]:
import numpy as np
print (np.__version__)
!pip install numpy==1.15.4 # to avoid bug loading hdf file "ValueError: cannot set WRITEABLE flag to True of this array"
print (np.__version__)


In [0]:
#### Reading file from Google Drive
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# download dataset 
#directory download = drive.CreateFile({'id': '1TuvyEky9lCcc5ybC6nTl5zzkIm8p8AVO'})
#download.GetContentFile("dataWW")

#download three file 
download = drive.CreateFile({'id': '1RN6BVBH4A3v_BBKguiEDJQ2z_pLNWHL_'})
download.GetContentFile("dataWW_qq.hdf")
download = drive.CreateFile({'id': '12kRnrvsluiBvXqvNOk2lb0Agna13lSzV'})
download.GetContentFile("dataWW_ggH.hdf")
download = drive.CreateFile({'id': '1nsGp13Mw3CzjtON---j1cWQziuFpkoEp'})
download.GetContentFile("dataWW_VBFH.hdf")



!ls -lrt

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from IPython import display
from IPython.display import display, HTML
%matplotlib inline
import time
pd.set_option('display.max_columns', None) # to see all columns of df.head()
np.random.seed(31415) # set the random seed

#!pip install pytables # to read hdf5
#!pip install xgboost
#!pip install lightgbm # not sure it loads the .so
#!conda install --yes lightgbm # better run in separate window

In [0]:
# some utilities
from math import sqrt
from math import log


def amsasimov(s,b): # asimove significance
        if b<=0 or s<=0:
            return 0
        try:
            return sqrt(2*((s+b)*log(1+float(s)/b)-s))
        except ValueError:
            print(1+float(s)/b)
            print (2*((s+b)*log(1+float(s)/b)-s))
        #return s/sqrt(s+b)

#
def compare_train_test(y_pred_train, y_train, y_pred, y_test, high_low=(0,1), bins=30, xlabel="", ylabel="Arbitrary units", title="", weights_train=np.array([]), weights_test=np.array([])):
    if weights_train.size != 0:
        weights_train_signal = weights_train[y_train == 1]
        weights_train_background = weights_train[y_train == 0]
    else:
        weights_train_signal = None
        weights_train_background = None
    plt.hist(y_pred_train[y_train == 1],
                 color='r', alpha=0.5, range=high_low, bins=bins,
                 histtype='stepfilled', normed=True,
                 label='S (train)', weights=weights_train_signal) # alpha is transparancy
    plt.hist(y_pred_train[y_train == 0],
                 color='b', alpha=0.5, range=high_low, bins=bins,
                 histtype='stepfilled', normed=True,
                 label='B (train)', weights=weights_train_background)

    if weights_test.size != 0:
        weights_test_signal = weights_test[y_test == 1]
        weights_test_background = weights_test[y_test == 0]
    else:
        weights_test_signal = None
        weights_test_background = None
    hist, bins = np.histogram(y_pred[y_test == 1],
                                  bins=bins, range=high_low, normed=True, weights=weights_test_signal)
    scale = len(y_pred[y_test == 1]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    #width = (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.errorbar(center, hist, yerr=err, fmt='o', c='r', label='S (test)')

    hist, bins = np.histogram(y_pred[y_test == 0],
                                  bins=bins, range=high_low, normed=True, weights=weights_test_background)
    scale = len(y_pred[y_test == 0]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    #width = (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.errorbar(center, hist, yerr=err, fmt='o', c='b', label='B (test)')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc='best')


In [0]:
#load signal, background data
#more info on dataset http://opendata.atlas.cern/books/current/openatlasdatatools/_book/simulated_data_details.html
# dataset ID qq : 105985, ggH : 160155, vbfh : 160205
# cross section  qq: 12.42 pb, ggH: 13.17 pb , VBFH : 1.617 pb
# unfortunately cross section in the document are wrong
qq_cross_section=1242 # deliberately wrong to increase background level
ggh_cross_section=13.17*0.5 # deliberately wrong to 
vbfh_cross_section=1.617*0.5 # deliberately wrong to 



# total integrated luminosity ATLAS 2012 : fb^-1
luminosity=21 
# even more info in http://opendata.atlas.cern/books/current/openatlasdatatools/_book/glossary.html 

#if fail to load, might be due to numpy version, see first cell
qq_events = pd.read_hdf("dataWW_qq.hdf","qq",WRITEABLE=False ) #quarks (qq background) http://opendata.cern.ch/record/3800
ggh_events = pd.read_hdf("dataWW_ggH.hdf","ggH",WRITEABLE=False ) # gluons fusion http://opendata.cern.ch/record/3825
vbfh_events = pd.read_hdf("dataWW_VBFH.hdf","VBFH",WRITEABLE=False ) #Vector Boson Fusion http://opendata.cern.ch/record/3826
vbfh_events.columns


# load signal, background data

In [0]:
nvbfkeep=int(ggh_events.shape[0]*0.1232)
print ("keep only ",nvbfkeep, " vbfh events to respect vbf H/ gg H ratio")
vbfh_events=vbfh_events[0:nvbfkeep]


qq_events["class"] = 0   # background category
ggh_events["class"] = 1  # signal category
vbfh_events["class"] = 1  # signal category
h_events = pd.concat([ggh_events, vbfh_events]) # merge the two higgs dataset
del ggh_events  # avoid mistakes later
del vbfh_events  # avoid mistakes later

In [0]:
h_events.describe()

In [0]:
h_events.head()

In [0]:

vbfh_cross_section=1.617
h_cross_section=ggh_cross_section+vbfh_cross_section


print ("before normalisation")
# in principle should multiply all the scaleFactor (then weights will vary event by event)
class_weights = (qq_events.mcWeight.sum(), h_events.mcWeight.sum()) 
print("total class weights",class_weights)


class_nevents = (len(qq_events.index), len(h_events.index))
print ("total class number of events",class_nevents)

qq_weight=qq_cross_section*luminosity/qq_events.shape[0]*qq_events.mcWeight
qq_events["weight"]=qq_weight
h_weight=h_cross_section*luminosity/h_events.shape[0]*h_events.mcWeight
h_events["weight"]=h_weight

print ("after normalisation")
# in principle should multiply all the scaleFactor (then weights will vary event by event)
total_weights = (qq_events.weight.sum(), h_events.weight.sum()) 

print("total total weights",total_weights)










In [0]:
fulldata = pd.concat([qq_events, h_events]) # merge the datasets
fulldata = fulldata.sample(frac=1).reset_index(drop=True) #shuffle the events
fulldata.head(10)




## Event selection

In [0]:
print (fulldata.shape)
fulldata=fulldata[fulldata.lep_n==2]
print (fulldata.shape)



In [0]:
fulldata.head()

In [0]:
#DR replace dummy -999 by -10 because of XGboost hist bug
fulldata.replace(-999,-7,inplace=True)
fulldata.head()

In [0]:
#hide class in separate vector
#WARNING : there should be no selection nor shuffling later on !
target = fulldata["class"]
del fulldata["class"]

#hide weight in separate vector
weights = fulldata["weight"]
del fulldata["weight"]
fulldata.shape

# DO NOT MODIFY ANYTHING ABOVE

In [0]:
# always rerun exercises from this cell
# for simplicity of the exercise only keep some features
data=fulldata[["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_phi_0', 'lep_phi_1','jet_n','jet_eta_0', 'jet_eta_1']]
#data=fulldata[["met_et","met_phi","lep_pt_0","lep_pt_1",'lep_eta_0', 'lep_eta_1', 'lep_phi_0', 'lep_phi_1','jet_n','jet_pt_0',
#       'jet_pt_1', 'jet_eta_0', 'jet_eta_1', 'jet_phi_0', 'jet_phi_1']]
print (data.shape)
data.head()




In [0]:
plt.figure()

data[target==0].hist(figsize=(15,12),color='b')
data[target==1].hist(figsize=(15,12),color='r')

plt.show()

# Feature engineering

In [0]:

#data["lep_deltaphi"]=np.abs(np.mod(data["lep_phi_0"]-data["lep_phi_1"]+3*np.pi,2*np.pi)-np.pi)
#data["lep_deltaphi"]=data["lep_phi_0"]-data["lep_phi_1"]

print (data.shape)
data.head()



## Transformation of the features

In [0]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
train_size = 0.75 # fraction of sample used for training
X_train, X_test, y_train, y_test, weights_train, weights_test = \
    train_test_split(data, target, weights, train_size=train_size)
#reset index for dataseries, not needed for ndarray (X_train, X_test)
y_train, y_test, weights_train, weights_test = \
    y_train.reset_index(drop=True),y_test.reset_index(drop=True), \
    weights_train.reset_index(drop=True), weights_test.reset_index(drop=True)
print (X_train.shape)
print (y_train.shape)
print (weights_train.shape)
print (X_test.shape)
print (y_test.shape)
print (weights_test.shape)





#scale to mean 0 and variance 1
scaler = StandardScaler()
#scaler.fit(data)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  # applied the transformation calculated the line above


class_weights_train = (weights_train[y_train == 0].sum(), weights_train[y_train == 1].sum())
print ("class_weights_train:",class_weights_train)
for i in range(len(class_weights_train)):
    weights_train[y_train == i] *= max(class_weights_train)/ class_weights_train[i] #equalize number of background and signal event
    weights_test[y_train == i] *= 1/(1-train_size) # increase test weight to compensate for sampling
    
print ("Test : total weight bkg", weights_test[y_test == 0].sum())
print ("Test : total weight sig", weights_test[y_test == 1].sum())
print ("Train : total weight bkg", weights_train[y_train == 0].sum())
print ("Train : total weight sig", weights_train[y_train == 1].sum())



    


# Testing BDT

## XGBoost

In [0]:
np.random.seed(31415) # set the random seed

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score # for binary classification if x > 0.5 -> 1 else -> 0
xgb = XGBClassifier(tree_method="hist")
#xgb = XGBClassifier(tree_method="hist",max_depth=12) # HPO, check on the web for other parameters
# not a bad idea to check for bugs without hist


starting_time = time.time( )
xgb.fit(X_train, y_train.values, sample_weight=weights_train.values)
training_time = time.time( ) - starting_time
print("Training time:",training_time)

y_pred_xgb = xgb.predict_proba(X_test)[:,1]
y_pred_xgb = y_pred_xgb.ravel()
y_pred_train_xgb = xgb.predict_proba(X_train)[:,1].ravel()
auc_test_xgb = roc_auc_score(y_true=y_test, y_score=y_pred_xgb)
print("auc test:",auc_test_xgb)
print ("auc train:",roc_auc_score(y_true=y_train.values, y_score=y_pred_train_xgb,))

int_pred_test_sig_xgb = [weights_test[(y_test ==1) & (y_pred_xgb > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]
int_pred_test_bkg_xgb = [weights_test[(y_test ==0) & (y_pred_xgb > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]

vamsasimov_xgb = [amsasimov(sumsig,sumbkg) for (sumsig,sumbkg) in zip(int_pred_test_sig_xgb,int_pred_test_bkg_xgb)]
significance_xgb = max(vamsasimov_xgb)
Z = significance_xgb
print("Z:",Z)
# To save model
xgb.save_model("XGBoost.model")

In [0]:
#gridSearchCV for advanced HPO, check on the web for other parameters

if False:
    from sklearn.model_selection import GridSearchCV

    #param_list = {'max_depth': [3, 6, 7], 'subsample': [0.7, 1],
    #                    'learning_rate': [0.05, 0.3], 'n_estimators': [10, 50, 200]}
    param_list_XGB = {'max_depth': [6,10], 'subsample': [0.7,1],
                        'learning_rate': [0.05, 0.3], 'max_leaves': [50, 200]}


    gsearch1 = GridSearchCV(estimator = XGBClassifier(), 
    param_grid = param_list_XGB, scoring='roc_auc',n_jobs=4,iid=False, cv=3)
    gsearch1.fit(X_train,y_train, weights_train)
    print (gsearch1.best_params_)
    print (gsearch1.best_score_)

    y_pred_gs = gsearch1.predict_proba(X_test)[:,1]
    roc_auc_score(y_true=y_test, y_score=y_pred_gs, sample_weight=weights_test)

## LightGBM

In [0]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score # for binary classification if x > 0.5 -> 1 else -> 0
#gbm = lgb.LGBMClassifier()
gbm = lgb.LGBMClassifier()
# gbm = lgb.LGBMClassifier(max_depth=12) # HPO, check on the web for other parameters


starting_time = time.time( )

gbm.fit(X_train, y_train.values,sample_weight=weights_train.values)
#gbm.fit(X_train, y_train.values) #ma


training_time = time.time( ) - starting_time
print("Training time:",training_time)

y_pred_gbm = gbm.predict_proba(X_test)[:,1]
y_pred_gbm = y_pred_gbm.ravel()
y_pred_train_gbm = gbm.predict_proba(X_train)[:,1].ravel()
auc_test_gbm = roc_auc_score(y_true=y_test, y_score=y_pred_gbm)
print("auc test:",auc_test_gbm)
print ("auc train:",roc_auc_score(y_true=y_train.values, y_score=y_pred_train_gbm,))

int_pred_test_sig_gbm = [weights_test[(y_test ==1) & (y_pred_gbm > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]
int_pred_test_bkg_gbm = [weights_test[(y_test ==0) & (y_pred_gbm > th_cut)].sum() for th_cut in np.linspace(0,1,num=50)]

vamsasimov_gbm = [amsasimov(sumsig,sumbkg) for (sumsig,sumbkg) in zip(int_pred_test_sig_gbm,int_pred_test_bkg_gbm)]
significance_gbm = max(vamsasimov_gbm)
Z = significance_gbm
print("Z:",Z)
# To save model
gbm.booster_.save_model("LightGBM.model")

In [0]:
#gridSearchCV for advanced HPO
if False:
    from sklearn.model_selection import GridSearchCV
    #param_list_GBM = {'max_depth': [3, 6, 7],
    #                     'learning_rate': [0.01, 0.05, 0.1, 0.3, 1], 'n_estimators': [10, 20, 40, 50, 200]}
    param_list_GBM = {'max_depth': [6,10], 
                        'learning_rate': [0.05, 0.3], 'n_estimators': [10, 200]}


    gsearch1 = GridSearchCV(estimator = XGBClassifier(), 
    param_grid = param_list_GBM, scoring='roc_auc',n_jobs=4,iid=False, cv=2)
    gsearch1.fit(X_train,y_train, weights_train)
    print (gsearch1.best_params_)
    print (gsearch1.best_score_)

    y_pred_gs = gsearch1.predict_proba(X_test)[:,1]
    roc_auc_score(y_true=y_test, y_score=y_pred_gs, sample_weight=weights_test)

In [0]:
print('Best significance found are:')
print('LightGBM: ', significance_gbm)
print('XGBoost : ', significance_xgb)
print('Best auc train found are:')
print('LightGBM: ', roc_auc_score(y_true=y_train.values, y_score=y_pred_train_gbm,))
print('XGBoost: ', roc_auc_score(y_true=y_train.values, y_score=y_pred_train_xgb,)) 

## Some nice plots 

In [0]:
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages('LightGBM_XGBoost.pdf')

In [0]:

compare_train_test(y_pred_train_xgb, y_train, y_pred_xgb, y_test, xlabel="XGboost score", title="XGboost")#, weights_train=weights_train.values, weights_test=weights_test.values)
#plt.savefig(new_dir + "/Score_BDT_XGBoost_Hist.pdf")
pdf.savefig()
plt.show()
compare_train_test(y_pred_train_gbm, y_train, y_pred_gbm, y_test, xlabel="LightGBM score", title="LightGBM")#, weights_train=weights_train.values, weights_test=weights_test.values)
#plt.savefig(new_dir + "/Score_BDT_LightGBM.pdf")
pdf.savefig()
plt.show()

In [0]:
from sklearn.metrics import roc_curve
lw = 2

fpr_gbm,tpr_gbm,_ = roc_curve(y_true=y_test, y_score=y_pred_gbm,)#,sample_weight=weights_test.values)
fpr_xgb,tpr_xgb,_ = roc_curve(y_true=y_test, y_score=y_pred_xgb,)#,sample_weight=weights_test.values)
plt.plot(fpr_gbm, tpr_gbm, color='darkorange',lw=lw, label='LightGBM (AUC  = {})'.format(np.round(auc_test_gbm,decimals=2)))
plt.plot(fpr_xgb, tpr_xgb, color='darkgreen',lw=lw, label='XGBoost (AUC  = {})'.format(np.round(auc_test_xgb,decimals=2)))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
#import os
#new_dir = "Plots/Comparing" 
#if not os.path.isdir(new_dir):
#    os.mkdir(new_dir)
#plt.savefig(new_dir + "/ROC_comparing.pdf")
pdf.savefig()
plt.show() # blue line = random classification -> maximize true positive rate while miniize false positive rate

In [0]:
plt.plot(np.linspace(0,1,num=50),vamsasimov_gbm, label='LightGBM (Z = {})'.format(np.round(significance_gbm,decimals=2)))
plt.plot(np.linspace(0,1,num=50),vamsasimov_xgb, label='XGBoost (Z = {})'.format(np.round(significance_xgb,decimals=2)))

plt.title("BDT Significance")
plt.xlabel("Threshold")
plt.ylabel("Significance")
plt.legend()
#plt.savefig(new_dir + "/Significance_comparing.pdf")
pdf.savefig()
plt.show()

In [0]:
plt.bar(data.columns.values, xgb.feature_importances_)
plt.xticks(rotation=90)
plt.title("Feature importances XGBoost Hist")
#plt.savefig(new_dir + "/VarImp_BDT_XGBoost_Hist.pdf",bbox_inches='tight')
#pdf.savefig(bbox_inches='tight')
plt.show()
plt.bar(data.columns.values, gbm.feature_importances_)
plt.xticks(rotation=90)
plt.title("Feature importances LightGBM")
#plt.savefig(new_dir + "/VarImp_BDT_LightGBM.pdf",bbox_inches='tight')
#pdf.savefig(bbox_inches='tight')
plt.show()
pdf.close()

# Permutation importance

In [0]:
#a bit slow
#!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance
file = open('html_original.html', 'w')
file.write(HTML('<h1>Permutation importances XGBoost</h1>').data)
perm_xgb = PermutationImportance(xgb, random_state=1).fit(X_test, y_test)#, sample_weight=weights_test.values)
html_xgb = eli5.show_weights(perm_xgb, feature_names = data.columns.values).data
#with open('html_xgb.html', 'w') as f:
#    f.write(HTML('<h1>Permutation importances XGBoost Hist</h1>').data)
#    f.write(html_xgb)
file.write(html_xgb)
perm_gbm = PermutationImportance(gbm, random_state=1).fit(X_test, y_test)#, sample_weight=weights_test.values)
html_gbm = eli5.show_weights(perm_gbm, feature_names = data.columns.values).data
#with open('html_gbm.html', 'w') as f:
#    f.write(HTML('<h1>Permutation importances LightGBM</h1>').data)
#    f.write(html_gbm)
file.write(HTML('<h1>Permutation importances LightGBM</h1>').data)
file.write(html_gbm)
print ("Permutation importances XGBoost")
display(eli5.show_weights(perm_xgb, feature_names = data.columns.values))
print ("Permutation importances LightGBM")
display(eli5.show_weights(perm_gbm, feature_names = data.columns.values))
file.close()